<a href="https://colab.research.google.com/github/VinoJose/Tensorflow/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

## Check the GPU instance

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-891d36b5-6e6e-d1cb-c4dd-140c4197c3cc)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-06-20 10:12:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-20 10:12:07 (106 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-20 10:12:11--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 173.194.202.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-06-20 10:12:11 (136 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle the training dataset
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
 # What does the test dataset look like?
 test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  #print(row)
  _, text, target = row
  print(f"Target:{target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("----\n")

Target:1 (real disaster)
Text:
Typhoon Soudelor approaches after 7 killed 2 missing in floods in Philippines http://t.co/nJMiDySXoF via @abc7chicago

----

Target:1 (real disaster)
Text:
Lol
Look how tough some people talk that live in some of the most gang infested cities in America
Your mouth isn't a weapon
You fucking wimp

----

Target:0 (not real disaster)
Text:
@_itsmegss_ I think it is. well it's bloody barking now

----

Target:0 (not real disaster)
Text:
Cooool :)

----

Target:0 (not real disaster)
Text:
Electricity cant stop scofield. Nigga survived a hotbox in SONA

----



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)


In [12]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

### Converting text to numbers

### Text vectorization (Tokenization)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use default text vertorization parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [15]:
# Find the average number of words in the training tweets
round(sum(len(i.split()) for i in train_sentences)/len(train_sentences))

15

In [16]:
# Setup the text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max number of words in our sequence

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training set and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original sentence:{random_sentence} \n Vectorized sentence:")
text_vectorizer([random_sentence])

Original sentence:http://t.co/FhI4qBpwFH @FredOlsenCruise Please take the #FaroeIslands off your itinerary until the mass murder of dolphins &amp; whales stops. 
 Vectorized sentence:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,  170,  167,    2, 3860,  102,   33,    1,  293,    2,
         157,  315,    6,    1]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [21]:
print(f"The number of words in the vocab: {len(words_in_vocab)}")
print(f"The top 5 common words in the vocab: {top_5_words}")
print(f"The bottom 5 common words in the vocab: {bottom_5_words}")

The number of words in the vocab: 10000
The top 5 common words in the vocab: ['', '[UNK]', 'the', 'a', 'in']
The bottom 5 common words in the vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding layer

In [22]:
 from tensorflow.keras import layers

 embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)
 embedding

In [23]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original sentence: \n{random_sentence} \n\nEmbedded version:")

# Embed the random sentence (turn it into dence vectors of fixed size)
sample_embed = embedding(text_vectorizer(random_sentence))
sample_embed

Original sentence: 
Listening to my grandad talk about his holiday is great but I don't want to be inundated with train photos ???? 

Embedded version:


<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.01182051, -0.02228912, -0.02459766, ..., -0.01257948,
         0.03504293, -0.0329787 ],
       [-0.02131767, -0.00931356, -0.03375188, ..., -0.04859055,
        -0.01694966, -0.02065537],
       [ 0.00361923, -0.02062323, -0.02519637, ..., -0.01440344,
        -0.01891053, -0.0077152 ],
       ...,
       [-0.01458334, -0.00177431, -0.02585986, ...,  0.01348888,
        -0.04217473, -0.01157327],
       [ 0.04307831, -0.03110744,  0.04293877, ..., -0.00121139,
         0.01273053, -0.01864462],
       [-0.02131767, -0.00931356, -0.03375188, ..., -0.04859055,
        -0.01694966, -0.02065537]], dtype=float32)>

## Modeling a text dataset (running a series of experiments)

Now we know how to turn text data to series of numbers. We can now start a series of model building experiments.

We'll start with a baseline and move on from there.

### Model 0: Getting a baseline

As with all machine learning experiments, let's create a baseline model. We can create baseline model with SKlearn's Naive Bayes using TF-IDF


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())  
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Out baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Out baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates the model accuracy, precision, recall and f1 score for a binary classifier
  """
  model_accuracy = accuracy_score(y_true, y_pred)

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average= 'weighted')
  model_results = {'accuracy' : model_accuracy,
                   'precision' : model_precision,
                   'recall' : model_recall,
                   'f1' : model_f1}
  return model_results
